In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.0.0


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
try:
    # Restrict TensorFlow to only use the second GPU
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:
    print(e)

2 Physical GPUs, 1 Logical GPUs


In [5]:
def create_vocab(text):
    # The unique characters in the file
    vocab = sorted(set(text))
    print ('{} unique characters'.format(len(vocab)))

    # Print chars in vocab
    print("Vocab: ", vocab)
    
    # VECTORIZE TEXT
    # Mapping from unique characters to indices
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)

    print("Character to index map: ", char2idx)
    print("Reverse map: ", idx2char)
    
    return vocab, char2idx, idx2char
    

def process_shakeaspeare_dataset():
    # By default saves file to ~/.keras/datasets/fname. ~/.keras is the cache_dir and if file is already present there, then it is not downloaded again.
    path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
    
    # Read, then decode for py2 compat.
    text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
    # length of text is the number of characters in it
    print ('Length of text: {} characters'.format(len(text)))

    # Peek at data
    print("Sample text: \n{} \n".format(text[:100]))
    v, c2id, id2c = create_vocab(text)
    return text, v, c2id, id2c

In [6]:
text, vocab, char2idx, idx2char = process_shakeaspeare_dataset()

Length of text: 1115394 characters
Sample text: 
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 

65 unique characters
Vocab:  ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Character to index map:  {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52

### Prediction Task
Given a sequence of characters, what is the most probable next character? 

In [7]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

In [8]:
tf.data.Dataset.from_tensor_slices??

In [9]:
# encode text 
text_as_int = np.array([char2idx[c] for c in text])

# create train data from the numpy array
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
# We convert the individual chars to sequences of a desired size using batch method.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# print above
for item in sequences.take(5):
    print(repr("".join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
# For each sequence, duplicate and shift it by one character to form the target text.
# We use map method of the BatchDataset that applies a transformation to each element of the dataset and returns a new dataset containing transformed elements in the same order as input.
# We need to pass the transformation function as the argument

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
print(type(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [13]:
for input_example, target_example in dataset.take(1):
    print("Input data: ", repr("".join(idx2char[input_example.numpy()])))
    print("Target data: ", repr("".join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
BATCH_SIZE = 64

# BUFFER_SIZE elements will be placed in a buffer and shuffled amongst themselves.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [25]:
class CustomModel:
    def __init__(self, vocab, embedding_dim=256, rnn_units=1024, batch_size=BATCH_SIZE):
        tf.keras.backend.clear_session()
        self.vocab = vocab
        # Length of the vocabulary in chars
        self.vocab_size = len(vocab)

        # The embedding dimension
        self.embedding_dim = embedding_dim

        # Number of RNN units
        self.rnn_units = rnn_units
        
        self.model = self.build_model()
        self.optimizer = self.get_optimizer()
        
        self.batch_size = batch_size
        
        self.checkpoint_dir = './text_gen_train_checkpoints'
        self.checkpoint_prefix = os.path.join(self.checkpoint_dir, "ckpt_{epoch}")
        return
    
    def get_optimizer(self):
        return tf.keras.optimizers.Adam()
    
    def build_model(self, BATCH_SIZE):
        self.model = tf.keras.Sequential(
            [tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim, batch_input_shape=[BATCH_SIZE, None]), 
             tf.keras.layers.GRU(self.rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'), 
             tf.keras.layers.Dense(self.vocab_size)])
        self.model.summary()
        return self.model
    
    def get_entropy_loss(self, labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
    def get_callbacks(self):   
        ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self.checkpoint_prefix, save_weights_only=True)
        return [ckpt_callback]
    
    # tf.function decorator ensures that this is still callable like a function and also compiled as a graph to leverage benefits as faster execution, exporting to SavedModel, and run on GPU/TPU.
    @tf.function
    def train_step(self, inp, target):
        with tf.GradientTape() as tape:
            predictions = self.model(inp)
            loss = tf.reduce_mean(self.get_entropy_loss(target, predictions))
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_weights))
        
        return loss
    
    def train_model(self, dataset, EPOCHS=10):
        for epoch in range(EPOCHS):
            start = time.time()
            
            # initialize hidden state at the start of every epoch
            hidden = self.model.reset_states()
            
            for (batch_n, (inp, target)) in enumerate(dataset):
                loss = self.train_step(inp, target)
                
                if batch_n % 100 == 0:
                    template = "Epoch {} Batch {} Loss {}"
                    print(template.format(epoch+1, batch_n, loss))
            
            if (epoch + 1) % 5 == 0:
                self.model.save_weights(self.checkpoint_prefix.format(epoch=epoch))
            
            print('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
            print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
            
        self.model.save_weights(self.checkpoint_prefix.format(epoch=epoch))
        return
    
    def generate_text(self, start_string, text_length):
        """
        text_length: Number of characters to be generated
        """
        input_eval = [char2idx[s] for s in start_string]
        input_eval = tf.expand_dims(input_eval, 0)
        
        text_generated = []
        
        # Low temp results in more predictable text, and Higher temp results in more surprising text.
        temprature = 1.0
        
        # TODO: Why?
        self.model.reset_states()
        
        for i in range(text_length):
            predictions = self.model(input_eval)
            predictions = tf.squeeze(predictions, 0)
            
            # using a categorical distribution to predict the word returned by the model
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            
            # We pass the predicted word as the next input to the model along with the previous hidden state
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(idx2char[predicted_id])
            
        return (start_string + ''.join(text_generated))
    
    def restore_model(self):
        tf.train.latest_checkpoint(self.checkpoint_dir)
        self.model = self.build_model(batch_size=1)
    
    def test(self, dataset):
        self.train_model(dataset)
        #print(self.generate_text(u"ROMEO: ", 1000))
        return
    
    def test_generation(self):
        self.model.build(tf.TensorShape([1, None]))
        print(self.generate_text(u"ROMEO: ", 1000))
        return

In [26]:
cm = CustomModel(vocab)
cm.test(dataset)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1 Batch 0 Loss 4.1745924949646
Epoch 1 Batch 100 Loss 2.3393023014068604
Epoch 1 Loss 2.1580
Time taken for 1 epoch 6.7360146045684814 sec

Epoch 2 Batch 0 Loss 2.1526076793670654
Epoch 2 Batch 100 Loss 1.945540428161621
Epoch 2 Loss 1.8606
Time taken for 1 epoch 5.560896396636963 sec

Epoch 3 Batch 0 Loss 1.8079391717910767
Epoch 3 Batch 100 Loss 1.6943069696426392
E